In [1]:
!pip install tensorflow-recommenders --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.3 MB/s eta 0:00:00


In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
from tensorflow.keras import models, layers

In [8]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-ratings/incomplete.DIL43B_0.1.1/movielens-train.tfrecord*..…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.


In [9]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [10]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

**Building the Ranking Model**

In [11]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 64

    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

This model takes user ids and movie titles, and outputs a predicted rating:

In [12]:
RankingModel()((["42"], ["One Flew Over the Cuckoo's Nest (1975)"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00828474]], dtype=float32)>

In [13]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [14]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

**Training the Ranking Model**

In [15]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [16]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [17]:
model.fit(cached_train, epochs=20)

Epoch 1/20
10/10 [==============================] - 5s 65ms/step - root_mean_squared_error: 2.0518 - loss: 4.0456 - regularization_loss: 0.0000e+00 - total_loss: 4.0456
Epoch 2/20
10/10 [==============================] - 1s 55ms/step - root_mean_squared_error: 1.2140 - loss: 1.4623 - regularization_loss: 0.0000e+00 - total_loss: 1.4623
Epoch 3/20
10/10 [==============================] - 1s 52ms/step - root_mean_squared_error: 1.1229 - loss: 1.2605 - regularization_loss: 0.0000e+00 - total_loss: 1.2605
Epoch 4/20
10/10 [==============================] - 1s 55ms/step - root_mean_squared_error: 1.0956 - loss: 1.2013 - regularization_loss: 0.0000e+00 - total_loss: 1.2013
Epoch 5/20
10/10 [==============================] - 1s 55ms/step - root_mean_squared_error: 1.0728 - loss: 1.1519 - regularization_loss: 0.0000e+00 - total_loss: 1.1519
Epoch 6/20
10/10 [==============================] - 1s 57ms/step - root_mean_squared_error: 1.0488 - loss: 1.1018 - regularization_loss: 0.0000e+00 - total

**Evaluating the model**

In [18]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 5s 17ms/step - root_mean_squared_error: 0.9488 - loss: 0.9012 - regularization_loss: 0.0000e+00 - total_loss: 0.9012


{'root_mean_squared_error': 0.9488495588302612,
 'loss': 0.9048500657081604,
 'regularization_loss': 0,
 'total_loss': 0.9048500657081604}

Testing the ranking model by computing predictions for a set of movies and then rank these movies based on the predictions

In [19]:
test_ratings = {}
test_movie_titles = ["Shakespeare Behind Bars (2005)", "Dances with Wolves (1990)", "Out of Time (2003)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
Dances with Wolves (1990): [[4.101407]]
Shakespeare Behind Bars (2005): [[3.5114934]]
Out of Time (2003): [[3.5114934]]


**Saving the model**

In [20]:
tf.saved_model.save(model, "export")

In [21]:
loaded = tf.saved_model.load("export")

loaded({"user_id": np.array(["42"]), "movie_title": ["Speed (1994)"]}).numpy()

array([[3.9331765]], dtype=float32)